## Project 2 Baseline
Quora Project



# Load Data and Import Libraries

First, we load all data and necessary libraries. We split the `train` data into `miniTrain` and `miniTest` dataframes.



In [1]:
#Import Libraries
import os
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
import csv
import random

In [2]:
# # Code to read csv file from Google Drive into Colaboratory:
# !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials
# # Authenticate and create the PyDrive client.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# #Load test DF
# testLink = 'https://drive.google.com/file/d/1chKVOcXwNzGRe_hRxpGlq18NVaSRM8X9/view?usp=sharing'
# idTest = testLink.split('/')[-2]
# downloaded = drive.CreateFile({'id':idTest}) 
# downloaded.GetContentFile('test.csv')  
# test = pd.read_csv('test.csv')
# test

In [ ]:
# #Load train DF
# trainLink = 'https://drive.google.com/file/d/1fP-AWiUGLjLkL0c_u67HOUFmHOMO7qm2/view?usp=sharing'
# idTrain = trainLink.split('/')[-2]
# downloaded = drive.CreateFile({'id':idTrain}) 
# downloaded.GetContentFile('train.csv')  

In [3]:
# train = pd.read_csv('/Users/julian.hicks/Downloads/train.csv')
# train

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0
...,...,...,...
1306117,ffffcc4e2331aaf1e41e,What other technical skills do you need as a c...,0
1306118,ffffd431801e5a2f4861,Does MS in ECE have good job prospects in USA ...,0
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0


In [4]:
# #Split Training Data into MiniTrain and MiniTest (Test data has no labels so impossible to measure error from it)
# # random.seed(10)
# # miniTrain, miniDev, miniTest = train_test_split(train, test_size=0.2)

# miniTrain, miniDev, miniTest = \
#               np.split(train.sample(frac=1, random_state=42), 
#                        [int(.8*len(train)), int(.9*len(train))])

# miniTrain.to_csv('miniTrain.csv', index=False)

# miniTest.to_csv('miniTest.csv', index=False)

# miniDev.to_csv('miniDev.csv', index = False)

# print("miniTrain: ",len(miniTrain), "observations, ", round(len(miniTrain)/len(train)*100,0), "%;\n" ,
#       "miniDev: ",len(miniDev), "observations, ", round(len(miniDev)/len(train)*100,0), "%;\n" ,
#       "miniTest: ",len(miniTest), "observations, ", round(len(miniTest)/len(train)*100,0), "%;" )

miniTrain:  1044897 observations,  80.0 %;
 miniDev:  130612 observations,  10.0 %;
 miniTest:  130613 observations,  10.0 %;


In [41]:
columns = [
    "id",
    "label",
    "alpha",
    "text"
]

miniTrain = pd.read_csv('/Users/julian.hicks/Documents/mids/w207ml/W207FinalProject/datasets/train.tsv', names=columns, sep="\t")

miniTest = pd.read_csv('/Users/julian.hicks/Documents/mids/w207ml/W207FinalProject/datasets/test.tsv', names=columns, sep="\t")

miniDev = pd.read_csv('/Users/julian.hicks/Documents/mids/w207ml/W207FinalProject/datasets/dev.tsv', names=columns, sep="\t")

## `txtprp` Module 

In [40]:
from nltk.stem.porter import PorterStemmer
import numpy as np
import pandas as pd
import nltk as nltk
import re
import gensim 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from imblearn.over_sampling import SMOTE, RandomOverSampler

class TextObject(pd.DataFrame):
    """
    A collection of strings for processing. Is a DataFrame with specific 
    columns. 
    """

    def __init__(self):
        super().__init__(columns=[
            "original_text",
            "text",
            "label"
        ]
        )
        self._attrs["applied_transformations"] = []
        nltk.download("stopwords")
        nltk.download("punkt")

    def add_text(self, text_collection, labels=None):
        """
        Adds a collection of text to to the TextObject. Accepts an iterable or 
        array-like of all text strings. Optionally, accepts labels of the same 
        length. Modifies self to be a pandas DataFrame with column "text" and 
        potentially "labels".
        """
        self["original_text"] = text_collection
        self["text"] = self.original_text
        self._attrs["applied_transformations"] = []
        if type(labels) == type(None):
            return
        if len(labels) == len(self):
            self["label"] = labels

    # Text Processing Methods:

    def text_processing(self, function):
        """
        Applies a generic text processing function to the self.text field.
        """
        self.text = self.text.apply(function)
        self._attrs["applied_transformations"].append(function.__name__)

    def lower(self):
        self.text = self.text.str.lower()
        self._attrs["applied_transformations"].append("lower")
    
    def strip(self):
        self.text = self.text.str.strip()
        self._attrs["applied_transformations"].append("strip")

    def remove_single_digits(self):
        self.text = self.text.apply(lambda text: re.sub("([\d]+)", "", text))
        self._attrs["applied_transformations"].append("remove_single_digits")
    
    def remove_nonletter_chars(self):
        self.text = self.text.apply(lambda text: re.sub("[^A-Za-z0-9 \\n]", " ", text))
        self._attrs["applied_transformations"].append("remove_nonletter_chars")

    def stop_word_tokenize(self):
        def tokenize(text):
            self.stoplist = nltk.corpus.stopwords.words('english')
            finalTokens = []
            tokens = nltk.word_tokenize(text)
            for w in tokens:
                if (w not in self.stoplist):
                    finalTokens.append(w)
            text = " ".join(finalTokens)
            return text
        self.text = self.text.apply(tokenize)
        self._attrs["applied_transformations"].append("stop_word_tokenize")
    
    def stem_sentence(self):
        def stem(text):
            porter=PorterStemmer()
            token_words=nltk.tokenize.word_tokenize(text)
            token_words
            stem_sentence=[]
            for word in token_words:
                stem_sentence.append(porter.stem(word))
                stem_sentence.append(" ")
            return "".join(stem_sentence)
        self.text = self.text.apply(stem)
        self._attrs["applied_transformations"].append("stem_sentence")

    def lemmatize_sentence(self):
        def lem(text):
            wordnet_lemmatizer = nltk.stem.WordNetLemmatizer()
            #token_words
            token_words=nltk.tokenize.word_tokenize(text)
            lemm_sentence=[]
            for word in token_words:
                lemm_sentence.append(wordnet_lemmatizer.lemmatize(word))
                lemm_sentence.append(" ")
            return "".join(lemm_sentence)
        self.text = self.text.apply(lem)
        self._attrs["applied_transformations"].append("lemmatize_sentence")

    def process_all(self, lower=True, strip=True, remove_single_digits =True, 
        remove_nonletter_chars=True, stop_word_tokenize=True, stem_sentence=True,
        lemmatize_sentence=True):
        if lower:
            self.lower()
        if strip:
            self.strip()
        if remove_single_digits:
            self.remove_single_digits()
        if remove_nonletter_chars:
            self.remove_nonletter_chars()
        if stop_word_tokenize:
            self.stop_word_tokenize()
        if stem_sentence:
            self.stem_sentence()
        if lemmatize_sentence:
            self.lemmatize_sentence()


In [42]:
train = TextObject()
train.add_text(miniTrain.text, labels=miniTrain.label)
print(train.__dict__)
train

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/julian.hicks/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/julian.hicks/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


{'_is_copy': None, '_mgr': BlockManager
Items: Index(['original_text', 'text', 'label'], dtype='object')
Axis 1: RangeIndex(start=0, stop=1044897, step=1)
ObjectBlock: slice(0, 2, 1), 2 x 1044897, dtype: object
NumericBlock: slice(2, 3, 1), 1 x 1044897, dtype: int64, '_item_cache': {}, '_attrs': {'applied_transformations': []}, '_flags': <Flags(allows_duplicate_labels=True)>}


,original_text,text,label
0,What is the most effective classroom managemen...,What is the most effective classroom managemen...,0
1,Can I study abroad after 10th class from Bangl...,Can I study abroad after 10th class from Bangl...,0
2,How can I make friends as a college junior?,How can I make friends as a college junior?,0
3,How do I download free APK Minecraft: Pocket E...,How do I download free APK Minecraft: Pocket E...,0
4,"Like Kuvera, is ""Groww"" also a free online inv...","Like Kuvera, is ""Groww"" also a free online inv...",0
...,...,...,...
1044892,How is a video similar to ordinary graphics?,How is a video similar to ordinary graphics?,0
1044893,How does training with a speed bag make you a ...,How does training with a speed bag make you a ...,0
1044894,What marketing strategies are implemented to i...,What marketing strategies are implemented to i...,0
1044895,What are the characteristics of wireless adapt...,What are the characteristics of wireless adapt...,0


In [43]:
test = TextObject()
test.add_text(miniTest.text, labels=miniTest.label)
print(test.__dict__)
test

{'_is_copy': None, '_mgr': BlockManager
Items: Index(['original_text', 'text', 'label'], dtype='object')
Axis 1: RangeIndex(start=0, stop=130612, step=1)
ObjectBlock: slice(0, 2, 1), 2 x 130612, dtype: object
NumericBlock: slice(2, 3, 1), 1 x 130612, dtype: int64, '_item_cache': {}, '_attrs': {'applied_transformations': []}, '_flags': <Flags(allows_duplicate_labels=True)>}


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/julian.hicks/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/julian.hicks/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,original_text,text,label
0,Does my BDS degree from India count if I want ...,Does my BDS degree from India count if I want ...,0
1,Is there such a thing as an average face?,Is there such a thing as an average face?,0
2,Is Munich Volkshochschule a good choice to lea...,Is Munich Volkshochschule a good choice to lea...,0
3,What is The relationship between Texas and its...,What is The relationship between Texas and its...,0
4,Will the current mining equipment work with se...,Will the current mining equipment work with se...,0
...,...,...,...
130607,What's the best way to teach your daughter to ...,What's the best way to teach your daughter to ...,0
130608,What are similarities and differences between ...,What are similarities and differences between ...,0
130609,What is the best shoe cleaning product for tri...,What is the best shoe cleaning product for tri...,0
130610,Can I get a private medical college with 320 m...,Can I get a private medical college with 320 m...,0


In [44]:
dev = TextObject()
dev.add_text(miniDev.text, labels=miniDev.label)
print(dev.__dict__)
dev

{'_is_copy': None, '_mgr': BlockManager
Items: Index(['original_text', 'text', 'label'], dtype='object')
Axis 1: RangeIndex(start=0, stop=130613, step=1)
ObjectBlock: slice(0, 2, 1), 2 x 130613, dtype: object
NumericBlock: slice(2, 3, 1), 1 x 130613, dtype: int64, '_item_cache': {}, '_attrs': {'applied_transformations': []}, '_flags': <Flags(allows_duplicate_labels=True)>}


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/julian.hicks/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/julian.hicks/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,original_text,text,label
0,Where can I find best romantic shayaris?,Where can I find best romantic shayaris?,1
1,When we check the UPSC rank list; many of the ...,When we check the UPSC rank list; many of the ...,0
2,Is there is any web site to create search engi...,Is there is any web site to create search engi...,0
3,How can you earn $50 just clicking ads?,How can you earn $50 just clicking ads?,0
4,Which 3 branches of engineering have the most ...,Which 3 branches of engineering have the most ...,0
...,...,...,...
130608,How can the ViewSonic PA503S 3600 lumens SVGA ...,How can the ViewSonic PA503S 3600 lumens SVGA ...,0
130609,What are the biggest myths about Adolf Hitler?,What are the biggest myths about Adolf Hitler?,0
130610,What song played in the movie of the gifted wh...,What song played in the movie of the gifted wh...,0
130611,"What do Socrates, Thomas Kuhn and Karl Popper ...","What do Socrates, Thomas Kuhn and Karl Popper ...",0


In [45]:
train.lower()
train.strip()
train.remove_single_digits()
train.remove_nonletter_chars()
train.stop_word_tokenize()
train

<ipython-input-40-7fe0180fad7b>:69: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self.stoplist = nltk.corpus.stopwords.words('english')


,original_text,text,label
0,What is the most effective classroom managemen...,effective classroom management skill technique...,0
1,Can I study abroad after 10th class from Bangl...,study abroad th class bangladesh,0
2,How can I make friends as a college junior?,make friends college junior,0
3,How do I download free APK Minecraft: Pocket E...,download free apk minecraft pocket edition ios...,0
4,"Like Kuvera, is ""Groww"" also a free online inv...",like kuvera groww also free online investment ...,0
...,...,...,...
1044892,How is a video similar to ordinary graphics?,video similar ordinary graphics,0
1044893,How does training with a speed bag make you a ...,training speed bag make better fighter,0
1044894,What marketing strategies are implemented to i...,marketing strategies implemented increase sale...,0
1044895,What are the characteristics of wireless adapt...,characteristics wireless adapters,0


In [48]:
train.stem_sentence()
# train.lemmatize_sentence()
train

,original_text,text,label
0,What is the most effective classroom managemen...,effect classroom manag skill techniqu creat go...,0
1,Can I study abroad after 10th class from Bangl...,studi abroad th class bangladesh,0
2,How can I make friends as a college junior?,make friend colleg junior,0
3,How do I download free APK Minecraft: Pocket E...,download free apk minecraft pocket edit io iphon,0
4,"Like Kuvera, is ""Groww"" also a free online inv...",like kuvera groww also free onlin invest platf...,0
...,...,...,...
1044892,How is a video similar to ordinary graphics?,video similar ordinari graphic,0
1044893,How does training with a speed bag make you a ...,train speed bag make better fighter,0
1044894,What marketing strategies are implemented to i...,market strategi implement increa sale whirlpoo...,0
1044895,What are the characteristics of wireless adapt...,characterist wireless adapt,0


In [49]:
for m in train._attrs["applied_transformations"]:
    getattr(dev, m)()
    getattr(test, m)()

<ipython-input-40-7fe0180fad7b>:69: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self.stoplist = nltk.corpus.stopwords.words('english')


# Fasttext

We're going to try using fasttext, a pretrained model on this dataset. It requires its own text format for data, so we'll be writing out our data to a custom format. 

In [50]:
!pip install fasttext

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [51]:
import fasttext


In [65]:
# check to see what format fasttext wants

!wget https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz && tar xvzf cooking.stackexchange.tar.gz
!head cooking.stackexchange.txt

bash: wget: No such file or directory
head: cooking.stackexchange.txt: No such file or directory


In [53]:

def writeFT(text_object, dir):
    original = []
    processed = []
    for i in range(0,len(text_object)):
        original.append(f"__label__{text_object.label[i]} {text_object.original_text[i]}")
        processed.append(f"__label__{text_object.label[i]} {text_object.text[i]}")

    with open(f"{dir}_original.txt", "w") as file:
        file.write("\n".join(original))
    print(f"Original text written to {dir}_original.txt!")

    with open(f"{dir}_processed.txt", "w") as file:
        file.write("\n".join(processed))
    print(f"Processed text written to {dir}_processed.txt!")

In [54]:
writeFT(train, "train")
writeFT(dev, "dev")
writeFT(test, "test")

Original text written to train_original.txt!
Processed text written to train_processed.txt!
Original text written to dev_original.txt!
Processed text written to dev_processed.txt!
Original text written to test_original.txt!
Processed text written to test_processed.txt!


In [55]:
model_prep = fasttext.train_supervised(input="train_processed.txt", epoch=25)

In [56]:
model_prep.save_model("ft_model_prep.bin")

In [57]:
model_prep.predict("unversity best for me")

(('__label__0',), array([0.99987066]))

In [58]:
model_prep.test("dev_processed.txt")

(130613, 0.9494154486919373, 0.9494154486919373)

In [59]:
model_orig = fasttext.train_supervised(input="train_original.txt", epoch=25)

In [60]:
model_orig.save_model("ft_model_orig.bin")

In [61]:
model_orig.predict("What University would be best for me?")

(('__label__0',), array([0.99998903]))

In [62]:
model_orig.predict("Why are Republicans better than traitor democRATS?")

(('__label__1',), array([0.99333972]))

In [63]:
dev_preds_orig = []
for i in range(len(dev)):
    pred = model_orig.predict(re.sub('\\n','',dev.original_text[i]))
    dev_preds_orig.append(pred)

orig_class = []
orig_prob = []
for x in dev_preds_orig:
    orig_class.append(int(list(x[0][0]).pop()))
    orig_prob.append(x[1][0])

dev_preds_prep = []
for i in range(len(dev)):
    pred = model_prep.predict(re.sub('\\n','',dev.text[i]))
    dev_preds_prep.append(pred)

prep_class = []
prep_prob = []
for x in dev_preds_prep:
    prep_class.append(int(list(x[0][0]).pop()))
    prep_prob.append(x[1][0])


dev_preds = pd.DataFrame(dev)
dev_preds["orig_class"] = orig_class
dev_preds["orig_prob"]  = orig_prob
dev_preds["prep_class"] = prep_class
dev_preds["prep_prob"]  = prep_prob
dev_preds

,original_text,text,label,orig_class,orig_prob,prep_class,prep_prob
0,Where can I find best romantic shayaris?,find best romant shayari,1,0,1.000000,0,0.999221
1,When we check the UPSC rank list; many of the ...,check upsc rank list mani top rank holder youn...,0,0,0.981609,0,0.994295
2,Is there is any web site to create search engi...,web site creat search engin like googl,0,0,0.999752,0,0.994709
3,How can you earn $50 just clicking ads?,earn click ad,0,0,1.000009,0,0.999371
4,Which 3 branches of engineering have the most ...,branch engin scope today corpor environ futur,0,0,0.999996,0,0.999934
...,...,...,...,...,...,...,...
130608,How can the ViewSonic PA503S 3600 lumens SVGA ...,viewson pa lumen svga hdmi projector manufactu...,0,0,0.999990,0,0.999059
130609,What are the biggest myths about Adolf Hitler?,biggest myth adolf hitler,0,0,0.997512,0,0.957914
130610,What song played in the movie of the gifted wh...,song play movi gift mr bern carson littl,0,0,0.998563,0,0.965192
130611,"What do Socrates, Thomas Kuhn and Karl Popper ...",socrat thoma kuhn karl popper common,0,0,0.999546,0,0.987883


In [64]:
from sklearn.metrics import classification_report

print("Original Text: \n", classification_report(dev.label, orig_class))

print("Processed Text: \n", classification_report(dev.label, prep_class))

Original Text: 
               precision    recall  f1-score   support

           0       0.97      0.98      0.97    122465
           1       0.64      0.48      0.55      8148

    accuracy                           0.95    130613
   macro avg       0.80      0.73      0.76    130613
weighted avg       0.95      0.95      0.95    130613

Processed Text: 
               precision    recall  f1-score   support

           0       0.96      0.98      0.97    122465
           1       0.64      0.42      0.51      8148

    accuracy                           0.95    130613
   macro avg       0.80      0.70      0.74    130613
weighted avg       0.94      0.95      0.94    130613



In [66]:


calcDF = dev_preds[["original_text", "orig_class", "orig_prob", "label"]].copy()
calcDF = calcDF.rename(columns={
    "original_text": "text",
    "orig_class": "class",
    "orig_prob": "prob"
})

def df_pred(row):
    if row["class"]==1:
        return row.prob
    else:
        return 1-row.prob

calcDF["pred"] = calcDF.apply(df_pred, axis=1)
calcDF

,text,class,prob,label,pred
0,Where can I find best romantic shayaris?,0,1.000000,1,-1.192093e-07
1,When we check the UPSC rank list; many of the ...,0,0.981609,0,1.839083e-02
2,Is there is any web site to create search engi...,0,0.999752,0,2.480149e-04
3,How can you earn $50 just clicking ads?,0,1.000009,0,-8.940697e-06
4,Which 3 branches of engineering have the most ...,0,0.999996,0,4.172325e-06
...,...,...,...,...,...
130608,How can the ViewSonic PA503S 3600 lumens SVGA ...,0,0.999990,0,1.001358e-05
130609,What are the biggest myths about Adolf Hitler?,0,0.997512,0,2.488196e-03
130610,What song played in the movie of the gifted wh...,0,0.998563,0,1.437485e-03
130611,"What do Socrates, Thomas Kuhn and Karl Popper ...",0,0.999546,0,4.540682e-04


In [67]:
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score


def frange(start, stop, step):
  i = start
  while i < stop:
    yield i
    i += step

columnNames = ['threshold', 'f1Score', 'accuracy', 'recall', 'precision']
thresholdDF = pd.DataFrame(columns=columnNames)

for i in frange(0.05,1,0.05):
    calcDF['pred2'] = np.where(calcDF['pred'] > i, 1, 0)
    f1Score = f1_score(calcDF.label, calcDF.pred2)
    accuracy = accuracy_score(calcDF.label, calcDF.pred2)
    recall = recall_score(calcDF.label, calcDF.pred2)
    precision = precision_score(calcDF.label, calcDF.pred2)
    thresholdDF = thresholdDF.append({'threshold': i, 'f1Score': f1Score, 
                                      'accuracy':accuracy, 'recall':recall, 
                                      'precision':precision}, 
                                      ignore_index=True)

thresholdDF

,threshold,f1Score,accuracy,recall,precision
0,0.05,0.503270,0.898226,0.826460,0.361790
1,0.10,0.554029,0.923691,0.759818,0.435955
2,0.15,0.572934,0.934233,0.707167,0.481531
3,0.20,0.581738,0.940098,0.667771,0.515344
4,0.25,0.583352,0.944125,0.627025,0.545367
5,0.30,0.579830,0.946506,0.591679,0.568447
6,0.35,0.572487,0.947869,0.559524,0.586065
7,0.40,0.565584,0.949109,0.531051,0.604921
8,0.45,0.557186,0.950082,0.503436,0.623783
9,0.50,0.547135,0.950862,0.475822,0.643592


In [72]:
 #Find the optimum threshold and save it to variable to use further down creating predictions
 #Depending on the original random sample, optimum threshold will oscillate between 0.2 and 0.25
 
optimumThreshold_orig = thresholdDF.loc[thresholdDF['f1Score'].idxmax()][0]

optimumThreshold_orig

0.25

In [73]:


calcDF = dev_preds[["text", "prep_class", "prep_prob", "label"]].copy()
calcDF = calcDF.rename(columns={
#     "original_text": "text",
    "prep_class": "class",
    "prep_prob": "prob"
})

def df_pred(row):
    if row["class"]==1:
        return row.prob
    else:
        return 1-row.prob

calcDF["pred"] = calcDF.apply(df_pred, axis=1)
calcDF

,text,class,prob,label,pred
0,find best romant shayari,0,0.999221,1,0.000779
1,check upsc rank list mani top rank holder youn...,0,0.994295,0,0.005705
2,web site creat search engin like googl,0,0.994709,0,0.005291
3,earn click ad,0,0.999371,0,0.000629
4,branch engin scope today corpor environ futur,0,0.999934,0,0.000066
...,...,...,...,...,...
130608,viewson pa lumen svga hdmi projector manufactu...,0,0.999059,0,0.000941
130609,biggest myth adolf hitler,0,0.957914,0,0.042086
130610,song play movi gift mr bern carson littl,0,0.965192,0,0.034808
130611,socrat thoma kuhn karl popper common,0,0.987883,0,0.012117


In [74]:
def frange(start, stop, step):
  i = start
  while i < stop:
    yield i
    i += step

columnNames = ['threshold', 'f1Score', 'accuracy', 'recall', 'precision']
thresholdDF = pd.DataFrame(columns=columnNames)

for i in frange(0.05,1,0.05):
    calcDF['pred2'] = np.where(calcDF['pred'] > i, 1, 0)
    f1Score = f1_score(calcDF.label, calcDF.pred2)
    accuracy = accuracy_score(calcDF.label, calcDF.pred2)
    recall = recall_score(calcDF.label, calcDF.pred2)
    precision = precision_score(calcDF.label, calcDF.pred2)
    thresholdDF = thresholdDF.append({'threshold': i, 'f1Score': f1Score, 
                                      'accuracy':accuracy, 'recall':recall, 
                                      'precision':precision}, 
                                      ignore_index=True)

thresholdDF

,threshold,f1Score,accuracy,recall,precision
0,0.05,0.447523,0.862005,0.895925,0.298251
1,0.10,0.528545,0.908638,0.820938,0.389734
2,0.15,0.562972,0.926967,0.754050,0.449156
3,0.20,0.580363,0.936714,0.701522,0.494892
4,0.25,0.582523,0.942134,0.647153,0.529630
5,0.30,0.578020,0.945526,0.598061,0.559279
6,0.35,0.567279,0.947532,0.551301,0.584211
7,0.40,0.552582,0.948795,0.506873,0.607353
8,0.45,0.533165,0.949186,0.465145,0.624485
9,0.50,0.509976,0.949415,0.421944,0.644424


In [75]:
 #Find the optimum threshold and save it to variable to use further down creating predictions
 #Depending on the original random sample, optimum threshold will oscillate between 0.2 and 0.25
 
optimumThreshold = thresholdDF.loc[thresholdDF['f1Score'].idxmax()][0]

optimumThreshold

0.25

In [77]:
# Original Model is somewhat better, save down the threshold

from joblib import dump

dump(optimumThreshold_orig, "ft_ot.joblib")

['ft_ot.joblib']